In [85]:
# Task 4.1
import pymongo
import csv

def readFile(coll, filename="PATIENTS.txt"):
    # Initialisation
    file = open(filename, 'r') # Read mode
    reader = csv.reader(file)
    
    # Read from file
    for record in reader:
        document = {
            "PatientID": int(record[0]), 
            "Name": record[1], 
            "Appointment Date": record[2], 
            "Appointment Start Time": record[3], 
            "Doctor Assigned": record[4], 
            "Room Number": record[5], 
            "Amount Charged": round(float(record[6]), 2) # 2 decimal places
        }
        coll.insert_one(document)
    
    file.close()
    #print(data)
    for i in coll.find({}): print(i)

conn = pymongo.MongoClient('127.0.0.1', 27017)
conn.drop_database('Clinic')
db = conn.get_database('Clinic')
coll = db.get_collection('Patient')
readFile(coll)
conn.close()

{'_id': ObjectId('5f68cc536a16725b0a0e9231'), 'PatientID': 1, 'Name': 'Peter Tan', 'Appointment Date': '31112020', 'Appointment Start Time': '0800', 'Doctor Assigned': 'Doctor 1', 'Room Number': 'Room 2', 'Amount Charged': 100.0}
{'_id': ObjectId('5f68cc536a16725b0a0e9232'), 'PatientID': 2, 'Name': 'John Ng', 'Appointment Date': '31112020', 'Appointment Start Time': '0800', 'Doctor Assigned': 'Doctor 1', 'Room Number': 'Room 1', 'Amount Charged': 150.0}
{'_id': ObjectId('5f68cc536a16725b0a0e9233'), 'PatientID': 3, 'Name': 'Lily Soh', 'Appointment Date': '31112020', 'Appointment Start Time': '0900', 'Doctor Assigned': 'Doctor 2', 'Room Number': 'Room 2', 'Amount Charged': 50.0}
{'_id': ObjectId('5f68cc536a16725b0a0e9234'), 'PatientID': 4, 'Name': 'John Ng', 'Appointment Date': '31112020', 'Appointment Start Time': '1000', 'Doctor Assigned': 'Doctor 3', 'Room Number': 'Room 2', 'Amount Charged': 80.0}
{'_id': ObjectId('5f68cc536a16725b0a0e9235'), 'PatientID': 5, 'Name': 'Kyle Teo', 'Appo

In [48]:
# Task 4.2

### Validation #######################
def validateInput(query, validFunc):
    value = input(query)
    while not validFunc(value):
        print("The input is not valid")
        value = input(query)
    return value

def validDate(string):
    # Haven't valid the specific date (day)
    return (string.isdigit() and
            len(string) == 8 and
            0 <= int(string[0:2]) <= 31 and
            0 <= int(string[2:4]) <= 12)

def validTime(string):
    return (string.isdigit() and
            len(string) == 4 and
            0 <= int(string[0:2]) <= 23 and
            0 <= int(string[2:]) <= 59)

def validDoctor(string):
    return string in ['Doctor 1', 'Doctor 2', 'Doctor 3']

### Document ##############
def patientIDFromDoc(document):
    return document['PatientID']

def getNewPatientID(coll):
    data = []
    for record in coll.find({}):
        data.append(record)
    largestIdThing = max(data, key=patientIDFromDoc)
    return largestIdThing['PatientID'] + 1

def newPatientDocument(coll):
    document = {
            "PatientID": None, # Generate when done
            "Name": input("Enter Patient Name: "), 
            "Appointment Date": validateInput("Enter Appointment Date in DDMMYYYY:", validDate), 
            "Appointment Start Time": validateInput("Enter Appointment Start Time in HHMM, 24 hour format:", validTime),
            "Doctor Assigned": validateInput("Enter Doctor Assigned (Doctor 1, Doctor 2, Doctor 3): ", validDoctor),
            "Room Number": None, # record[5], 
            "Amount Charged": None #round(float(record[6]), 2) # 2 decimal places
        }
    document['PatientID'] = getNewPatientID(coll)
    coll.insert_one(document)
    
### Main ######################################################
def task4_2(): # Inser 1 patient
    conn = pymongo.MongoClient('127.0.0.1', 27017)
    db = conn.get_database('Clinic')
    coll = db.get_collection('Patient')
    #print(getNewPatientID(coll))
    if input() != "":
        newPatientDocument(coll)
    conn.close()
task4_2()

In [87]:
# Task 4.3 ## I don't understand what they want
from datetime import datetime, timedelta
def appointmentClashPerDoctor(coll, key):
    dateFormat = "%d%m%Y HHMM"
    dates = []
    appointments = {}
    cursor = coll.find()
    for record in cursor:
        if record["Doctor Assigned"] == key:
            #print(record)
            year = int(record['Appointment Date'][4:])
            month = int(record['Appointment Date'][2:4])
            day = min(30,int(record['Appointment Date'][0:2])) # Blame the data for not being accurate
            hour = int(record['Appointment Start Time'][0:2])
            minute = int(record['Appointment Start Time'][2:4])
            currDate = datetime(year=year, month=month, day=day, hour=hour, minute=minute)

            if appointments.get(currDate) == None:
                appointments[currDate] = record
                dates.append(currDate)
            else:
                # Double Booking
                if appointments[currDate] == record:
                    coll.delete_one(record)
                # Clash appointment: Move by 1 hour? What you want?
                else:
                    newDate = max(dates) + timedelta(hours=1)
                    newTimeString = newDate.strftime("%H%M")
                    newDateString = newDate.strftime("%d%m%Y")
                    og = dict(record)
                    coll.delete_one(og)
                    # Add new record
                    record["Appointment Date"] = newDateString
                    record["Appointment Start Time"] = newTimeString
                    appointments[newDate] = record
                    dates.append(newDate)
                    coll.insert_one(record)
                    print("Clash, moved", og, "to", newDate, newTimeString)
    
    print(dates)#, appointments)
                
        
### Main ######################################################
def task4_3(): # Inser 1 patient
    conn = pymongo.MongoClient('127.0.0.1', 27017)
    db = conn.get_database('Clinic')
    coll = db.get_collection('Patient')
    #print(getNewPatientID(coll))
    appointmentClashPerDoctor(coll, 'Doctor 1')
    appointmentClashPerDoctor(coll, 'Doctor 2')
    appointmentClashPerDoctor(coll, 'Doctor 3')
    conn.close()
task4_3()

Clash, moved {'_id': ObjectId('5f68cc536a16725b0a0e9232'), 'PatientID': 2, 'Name': 'John Ng', 'Appointment Date': '31112020', 'Appointment Start Time': '0800', 'Doctor Assigned': 'Doctor 1', 'Room Number': 'Room 1', 'Amount Charged': 150.0} to 2020-11-30 09:00:00 0900
[datetime.datetime(2020, 11, 30, 8, 0), datetime.datetime(2020, 11, 30, 9, 0), datetime.datetime(2020, 11, 30, 12, 0), datetime.datetime(2020, 12, 15, 8, 0), datetime.datetime(2020, 12, 22, 9, 0), datetime.datetime(2020, 12, 23, 11, 0), datetime.datetime(2020, 12, 24, 13, 0), datetime.datetime(2020, 12, 24, 14, 0), datetime.datetime(2020, 12, 28, 14, 0), datetime.datetime(2020, 12, 30, 9, 0)]
Clash, moved {'_id': ObjectId('5f68cc536a16725b0a0e9243'), 'PatientID': 4, 'Name': 'John Ng', 'Appointment Date': '24122020', 'Appointment Start Time': '1700', 'Doctor Assigned': 'Doctor 2', 'Room Number': 'Room 2', 'Amount Charged': 200.0} to 2020-12-24 18:00:00 1800
[datetime.datetime(2020, 11, 30, 9, 0), datetime.datetime(2020, 11

In [28]:
# Task 4.4

def key(a):
    return a[1]["Amount Charged"]

def topPayingPatients(coll):
    patients = {}
    cursor = coll.find({})
    #print(cursor.count())
    for record in cursor:
        patientID = record['PatientID']
        name = record['Name']
        amt = record['Amount Charged']
        
        if patients.get(patientID) == None:
            patients[patientID] = {"Name":name, "Amount Charged":amt}
        else:
            patients[patientID]["Amount Charged"] += amt
            
    data = list(patients.items())
    data.sort(key=key, reverse=True)
    return data

def task4_4():
    conn = pymongo.MongoClient('127.0.0.1', 27017)
    db = conn.get_database('Clinic')
    coll = db.get_collection('Patient')
    topPaying = topPayingPatients(coll)
    for i in range(3):
        print(topPaying[i])
    conn.close()

task4_4()

(6, {'Name': 'Richard Lee', 'Amount Charged': 670.0})
(3, {'Name': 'Lily Soh', 'Amount Charged': 610.0})
(4, {'Name': 'John Ng', 'Amount Charged': 570.0})


In [30]:
# Task 4.5

def duplicateName(coll):
    patients = {}
    duplicateNames = []
    cursor = coll.find({})
    #print(cursor.count())
    for record in cursor:
        patientID = record['PatientID']
        name = record['Name']
        amt = record['Amount Charged']
        
        if patients.get(name) == None:
            patients[name] = [patientID]
        elif patientID not in patients[name]:
            patients[name].append(patientID)
            duplicateNames.append(name)
            
    return patients, duplicateNames

def task4_5():
    conn = pymongo.MongoClient('127.0.0.1', 27017)
    db = conn.get_database('Clinic')
    coll = db.get_collection('Patient')
    patients, duplicateNames = duplicateName(coll)
    for name in duplicateNames:
        print(name, patients[name])
    conn.close()
    
task4_5()

John Ng [2, 4]


In [45]:
conn = pymongo.MongoClient('127.0.0.1', 27017)
db = conn.get_database('Clinic')
coll = db.get_collection('Patient')
for i in coll.find({}): print(i)
conn.close()

{'_id': ObjectId('5f68abe56a16725b0a0e9155'), 'PatientID': 1, 'Name': 'Peter Tan', 'Appointment Date': '31112020', 'Appointment Start Time': '0800', 'Doctor Assigned': 'Doctor 1', 'Room Number': 'Room 2', 'Amount Charged': 100.0}
{'_id': ObjectId('5f68abe66a16725b0a0e9156'), 'PatientID': 2, 'Name': 'John Ng', 'Appointment Date': '31112020', 'Appointment Start Time': '0800', 'Doctor Assigned': 'Doctor 1', 'Room Number': 'Room 1', 'Amount Charged': 150.0}
{'_id': ObjectId('5f68abe66a16725b0a0e9157'), 'PatientID': 3, 'Name': 'Lily Soh', 'Appointment Date': '31112020', 'Appointment Start Time': '0900', 'Doctor Assigned': 'Doctor 2', 'Room Number': 'Room 2', 'Amount Charged': 50.0}
{'_id': ObjectId('5f68abe66a16725b0a0e9158'), 'PatientID': 4, 'Name': 'John Ng', 'Appointment Date': '31112020', 'Appointment Start Time': '1000', 'Doctor Assigned': 'Doctor 3', 'Room Number': 'Room 2', 'Amount Charged': 80.0}
{'_id': ObjectId('5f68abe66a16725b0a0e9159'), 'PatientID': 5, 'Name': 'Kyle Teo', 'Appo